In [8]:
import time
import warnings
import numpy as np
from numpy import newaxis
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import matplotlib.pyplot as plt

def plot_results_multiple(predicted_data, true_data, prediction_len):
    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(true_data, label='True Data')
    print ('yo')
    #Pad the list of predictions to shift it in the graph to it's correct start
    for i, data in enumerate(predicted_data):
        padding = [None for p in range(i * prediction_len)]
        plt.plot(padding + data, label='Prediction')
        plt.legend()
    plt.show()
    
def load_data(filename, seq_len, normalise_window):
    f = open(filename, 'r').read()
    data = f.split('\n')

    sequence_length = seq_len + 1
    result = []
    for index in range(int(len(data) - sequence_length)):
        result.append(data[index: index + sequence_length])
    
    if normalise_window:
        result = normalise_windows(result)

    result = np.array(result)

    row = round(0.9 * result.shape[0])
    train = result[:int(row), :]
    np.random.shuffle(train)
    x_train = train[:, :-1]
    y_train = train[:, -1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  

    return [x_train, y_train, x_test, y_test]

def normalise_windows(window_data):
    normalised_data = []
    for window in window_data:
        normalised_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalised_data.append(normalised_window)
    return normalised_data

def build_model():
    model = Sequential()

    model.add(LSTM(
        input_dim=1,
        output_dim=50,
        return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(
        100,
        return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(
        output_dim=1))
    model.add(Activation("relu"))

    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop")
    print ("Compilation Time : "), time.time() - start
    return model

def predict_sequences_multiple(model, data, window_size, prediction_len):
    #Predict sequence of 50 steps before shifting prediction run forward by 50 steps
    prediction_seqs = []
    for i in range(int(len(data)/prediction_len)):
        curr_frame = data[i*prediction_len]
        predicted = []
        for j in range(prediction_len):
            predicted.append(model.predict(curr_frame[newaxis,:,:])[0,0])
            curr_frame = curr_frame[1:]
            curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
        prediction_seqs.append(predicted)
    return prediction_seqs

In [9]:
#Step 1 Load Data
X_train, y_train, X_test, y_test = load_data('sp500.csv', 22, True)

In [10]:
#model =build_model()

In [11]:
from keras.layers.advanced_activations import LeakyReLU, PReLU
def build_model2():
    model = Sequential()

    model.add(LSTM(
        input_shape=(22, 1),
        units=128,
        return_sequences=True))
    model.add(Dropout(rate=0.2,seed=1000))

    model.add(LSTM(
        input_shape=(22, 1),
        units=128,
        return_sequences=False))
    model.add(Dropout(rate=0.2,seed=1000))

    model.add(Dense(32,kernel_initializer="uniform",activation='relu'))        
    model.add(Dense(1,kernel_initializer="uniform",activation='linear'))
    #model.add(PReLU())
    #model.add(LeakyReLU(alpha=0.001))

    start = time.time()
    model.compile(loss="mse", optimizer="adam",metrics=['accuracy'])
    model.summary()
    return model

In [12]:
model=build_model2()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 22, 128)           66560     
_________________________________________________________________
dropout_3 (Dropout)          (None, 22, 128)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 202,305
Trainable params: 202,305
Non-trainable params: 0
_________________________________________________________________


In [13]:
# model.get_config()
# outputs = [layer.output for layer in model.layers] 
# inputs = [layer.input for layer in model.layers] 
# for x in inputs:
#     print(x)
# print("OUTPUT")
# for x in outputs:
#     print(x)

In [14]:
#Step 3 Train the model
model.fit(
    X_train,
    y_train,
    batch_size=512,
    epochs=300,
    validation_split=0.1)

Train on 3360 samples, validate on 374 samples
Epoch 1/300
3360/3360 [==============================] - 10s 3ms/step - loss: 0.0024 - acc: 0.0000e+00 - val_loss: 0.0027 - val_acc: 0.0000e+00
Epoch 2/300
3360/3360 [==============================] - 5s 1ms/step - loss: 0.0021 - acc: 0.0000e+00 - val_loss: 0.0020 - val_acc: 0.0000e+00
Epoch 3/300
3360/3360 [==============================] - 4s 1ms/step - loss: 0.0014 - acc: 0.0000e+00 - val_loss: 0.0015 - val_acc: 0.0000e+00
Epoch 4/300
3360/3360 [==============================] - 4s 1ms/step - loss: 9.7386e-04 - acc: 0.0000e+00 - val_loss: 0.0012 - val_acc: 0.0000e+00
Epoch 5/300
3360/3360 [==============================] - 4s 1ms/step - loss: 8.2645e-04 - acc: 0.0000e+00 - val_loss: 9.7283e-04 - val_acc: 0.0000e+00
Epoch 6/300
3360/3360 [==============================] - 4s 1ms/step - loss: 6.5971e-04 - acc: 0.0000e+00 - val_loss: 7.5382e-04 - val_acc: 0.0000e+00
Epoch 7/300
3360/3360 [==============================] - 5s 1ms/step - los

KeyboardInterrupt: 

In [ ]:
#Step 4 - Plot the predictions!
predictions = predict_sequences_multiple(model, X_test, 22, 30)
#print(predictions)
plot_results_multiple(predictions, y_test, 30)